<a href="https://colab.research.google.com/github/isshii/de4test/blob/test_ozawa/DeepXplore/DeepXplore_04_01_result_01_20200209_0046.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# 共通で使うパスなどの定義
# 共通の変数設定
# 共通フォルダパス

import datetime
import pytz
dt_now = datetime.datetime.now(pytz.timezone('Asia/Tokyo'))
dt_str = str(dt_now.strftime('%Y%m%d_%H%M'))
dt_str = "20200209_0046"

data_dir = "/content/gdrive/My Drive/ColabNotebooks/test4ai/data"
#data_imagenet = "/content/gdrive/My Drive/ColabNotebooks/test4ai/data/ImageNet"
data_mnist = "/content/gdrive/My Drive/ColabNotebooks/test4ai/data/MNIST"
#data_imagenet_seeds = "/content/gdrive/My Drive/ColabNotebooks/test4ai/data/ImageNet/seeds"
model_dir = "/content/gdrive/My Drive/ColabNotebooks/test4ai/model"
#output_dir = "/content/gdrive/My Drive/ColabNotebooks/test4ai/output"
output_dir = "/content/gdrive/My Drive/ColabNotebooks/test4ai/output/" + dt_str
tmp_dir = "/content/gdrive/My Drive/ColabNotebooks/test4ai/tmp"

In [0]:
# Goggle Drive つなぐ
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import sys
import argparse
import pandas as pd
pd.options.display.float_format = '{:.3f}'.format

%tensorflow_version 1.x
from keras.datasets import mnist
from keras.layers import Input
import imageio

from __future__ import print_function

from keras.datasets import mnist
from keras.layers import Convolution2D, MaxPooling2D, Input, Dense, Activation, Flatten
from keras.models import Model
from keras.utils import to_categorical
import os

# TensorFlowでGPUを使っているかのチェック：
# "device_type: "GPU" " があればOK
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

Using TensorFlow backend.


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 8047197658420005041, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 3129164647755118166
 physical_device_desc: "device: XLA_CPU device"]

In [0]:
parser = argparse.ArgumentParser(description='Main function for difference-inducing input generation in MNIST dataset')
parser.add_argument('transformation', help="realistic transformation type", choices=['light', 'occl', 'blackout'])
parser.add_argument('weight_diff', help="weight hyperparm to control differential behavior", type=float)
parser.add_argument('weight_nc', help="weight hyperparm to control neuron coverage", type=float)
parser.add_argument('step', help="step size of gradient descent", type=float)
parser.add_argument('seeds', help="number of seeds of input", type=int)
parser.add_argument('grad_iterations', help="number of iterations of gradient descent", type=int)
parser.add_argument('threshold', help="threshold for determining neuron activated", type=float)
parser.add_argument('-t', '--target_model', help="target model that we want it predicts differently",
                    choices=[0, 1, 2], default=0, type=int)
parser.add_argument('-sp', '--start_point', help="occlusion upper left corner coordinate", default=(0, 0), type=tuple)
parser.add_argument('-occl_size', '--occlusion_size', help="occlusion size", default=(10, 10), type=tuple)
noise_type = "light" #@param ["light", "occl", "blackout"]
weight_diff = "0.1" #@param {type:"string"}
weight_nc = "0.1" #@param {type:"string"}
step = "10" #@param {type:"string"}　#50だとほぼ白飛び
seeds = "10" #@param {type:"string"}
grad_iterations = "20" #@param {type:"string"}
threshold = "0.1" #@param {type:"string"}

args = parser.parse_args([noise_type, weight_diff, weight_nc, step, seeds, grad_iterations, threshold])

length = int(args.seeds)

df2_scale = pd.DataFrame()
df3_scale = pd.DataFrame()
df4_scale = pd.DataFrame()
for start_fig in range(10):
  #df1_scale = pd.read_csv(output_dir[:-14]+ "/01_fig" + str(start_fig) + "_seed" + str(length) +"_neuron.csv")
  df1_scale = pd.read_csv(output_dir + "/03_16_fig" + str(start_fig) + "_seed" + str(length) +"_neuron.csv")
  df2_scale = pd.concat([df2_scale, df1_scale])

  #df1_scale = pd.read_csv(output_dir[:-14]+ "/01_fig" + str(start_fig) + "_seed" + str(length) +"_index.csv")
  df1_scale = pd.read_csv(output_dir + "/03_16_fig" + str(start_fig) + "_seed" + str(length) +"_index.csv")
  df3_scale = pd.concat([df3_scale, df1_scale])

  #df1_scale = pd.read_csv(output_dir[:-14]+ "/01_fig" + str(start_fig) + "_seed" + str(length) +"_trace.csv")
  df1_scale = pd.read_csv(output_dir + "/03_16_fig" + str(start_fig) + "_seed" + str(length) +"_trace.csv")
  df4_scale = pd.concat([df4_scale, df1_scale])

tmp_list = list(df2_scale.iloc[:,0])
df2_scale.index = tmp_list
print(df2_scale.iloc[:,1:].head())

tmp_list = list(df3_scale.iloc[:,0])
df3_scale.index = tmp_list
print(df3_scale.iloc[:,1:].head())

tmp_list = list(df4_scale.iloc[:,0])
df4_scale.index = tmp_list
print(df4_scale.iloc[:,1:].head())

     ('block1_conv1', 0)  ...  ('predictions', 9)
0_1                0.005  ...               0.000
0_2                0.540  ...               0.000
0_3                0.357  ...               0.000
0_4                0.387  ...               0.000
0_5                0.537  ...               0.000

[5 rows x 52 columns]
     already_diff  found  not_found  ... index2        layer3  index3
0_1         0.000  4.000      0.000  ...  2.000           fc2  12.000
0_2         0.000  6.000      0.000  ... 79.000           fc1  89.000
0_3         0.000  0.000      1.000  ... 72.000  block2_conv1  14.000
0_4         0.000  0.000      1.000  ...  8.000           fc2  63.000
0_5         0.000  2.000      0.000  ...  4.000           fc1 116.000

[5 rows x 9 columns]
       ('block1_conv1', 0)  ...  ('predictions', 9)
0_1_0                0.084  ...               0.001
0_1_1                0.479  ...               0.000
0_1_2                0.228  ...               0.000
0_1_3                0.005 

In [0]:
df4_scale.head(20)

,Unnamed: 0,"('block1_conv1', 0)","('block1_conv1', 1)","('block1_conv1', 2)","('block1_conv1', 3)","('block1_pool1', 0)","('block1_pool1', 1)","('block1_pool1', 2)","('block1_pool1', 3)","('block2_conv1', 0)","('block2_conv1', 1)","('block2_conv1', 2)","('block2_conv1', 3)","('block2_conv1', 4)","('block2_conv1', 5)","('block2_conv1', 6)","('block2_conv1', 7)","('block2_conv1', 8)","('block2_conv1', 9)","('block2_conv1', 10)","('block2_conv1', 11)","('block2_pool1', 0)","('block2_pool1', 1)","('block2_pool1', 2)","('block2_pool1', 3)","('block2_pool1', 4)","('block2_pool1', 5)","('block2_pool1', 6)","('block2_pool1', 7)","('block2_pool1', 8)","('block2_pool1', 9)","('block2_pool1', 10)","('block2_pool1', 11)","('before_softmax', 0)","('before_softmax', 1)","('before_softmax', 2)","('before_softmax', 3)","('before_softmax', 4)","('before_softmax', 5)","('before_softmax', 6)","('before_softmax', 7)","('before_softmax', 8)","('before_softmax', 9)","('predictions', 0)","('predictions', 1)","('predictions', 2)","('predictions', 3)","('predictions', 4)","('predictions', 5)","('predictions', 6)","('predictions', 7)","('predictions', 8)","('predictions', 9)"
0_1_0,0_1_0,0.084,0.073,0.086,0.073,0.126,0.112,0.135,0.112,0.065,0.101,0.080,0.060,0.079,0.098,0.070,0.064,0.064,0.087,0.048,0.064,0.130,0.203,0.161,0.111,0.161,0.182,0.134,0.101,0.132,0.173,0.103,0.130,1.000,0.000,0.450,0.142,0.175,0.508,0.495,0.204,0.044,0.393,1.000,0.000,0.001,0.000,0.000,0.002,0.002,0.000,0.000,0.001
0_1_1,0_1_1,0.479,0.309,0.075,0.344,0.520,0.345,0.098,0.381,0.228,0.088,0.022,0.264,0.156,0.166,0.072,0.155,0.029,0.056,0.018,0.163,0.338,0.159,0.044,0.348,0.260,0.270,0.121,0.220,0.061,0.110,0.035,0.242,1.000,0.000,0.820,0.718,0.136,0.527,0.433,0.173,0.597,0.188,1.000,0.000,0.019,0.002,0.000,0.000,0.000,0.000,0.000,0.000
0_1_2,0_1_2,0.228,0.144,0.057,0.161,0.279,0.186,0.085,0.204,0.102,0.097,0.049,0.100,0.098,0.114,0.060,0.093,0.044,0.072,0.032,0.085,0.171,0.184,0.106,0.156,0.179,0.202,0.108,0.146,0.090,0.141,0.063,0.148,1.000,0.000,0.557,0.351,0.115,0.432,0.457,0.196,0.276,0.339,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
0_1_3,0_1_3,0.005,0.008,0.100,0.006,0.012,0.019,0.161,0.013,0.032,0.094,0.104,0.057,0.064,0.084,0.089,0.021,0.091,0.128,0.093,0.038,0.064,0.192,0.193,0.099,0.137,0.154,0.153,0.041,0.167,0.248,0.165,0.077,1.000,0.233,0.635,0.000,0.838,0.756,0.865,0.367,0.405,0.585,1.000,0.007,0.106,0.000,0.372,0.224,0.437,0.019,0.024,0.077
0_2_0,0_2_0,0.134,0.090,0.060,0.097,0.188,0.136,0.092,0.138,0.084,0.103,0.069,0.073,0.101,0.106,0.062,0.090,0.067,0.080,0.043,0.075,0.137,0.188,0.136,0.127,0.160,0.191,0.123,0.187,0.121,0.152,0.085,0.146,1.000,0.000,0.598,0.376,0.039,0.554,0.473,0.121,0.506,0.262,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
0_2_1,0_2_1,0.508,0.329,0.078,0.365,0.551,0.366,0.102,0.401,0.205,0.071,0.016,0.235,0.145,0.139,0.062,0.143,0.025,0.041,0.015,0.143,0.296,0.117,0.039,0.305,0.224,0.219,0.109,0.229,0.052,0.077,0.029,0.209,1.000,0.024,0.939,0.864,0.009,0.643,0.396,0.000,0.803,0.120,1.000,0.000,0.232,0.038,0.000,0.000,0.000,0.000,0.009,0.000
0_2_2,0_2_2,0.131,0.089,0.060,0.096,0.185,0.135,0.093,0.136,0.083,0.103,0.070,0.072,0.101,0.106,0.062,0.089,0.067,0.080,0.043,0.074,0.136,0.189,0.137,0.127,0.160,0.191,0.124,0.186,0.122,0.153,0.086,0.145,1.000,0.000,0.596,0.375,0.040,0.555,0.473,0.121,0.503,0.262,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
0_2_3,0_2_3,0.499,0.323,0.078,0.359,0.543,0.361,0.102,0.395,0.203,0.073,0.017,0.232,0.144,0.140,0.063,0.143,0.026,0.043,0.016,0.143,0.294,0.120,0.041,0.302,0.224,0.220,0.111,0.231,0.054,0.080,0.030,0.209,1.000,0.018,0.925,0.842,0.004,0.638,0.397,0.000,0.790,0.125,1.000,0.000,0.166,0.023,0.000,0.000,0.000,0.000,0.007,0.000
0_2_4,0_2_4,0.118,0.084,0.063,0.088,0.170,0.130,0.097,0.128,0.080,0.103,0.073,0.069,0.100,0.106,0.064,0.086,0.069,0.082,0.045,0.073,0.132,0.192,0.141,0.123,0.159,0.191,0.127,0.182,0.126,0.158,0.090,0.143,1.000,0.000,0.584,0.365,0.044,0.55

In [0]:
print(df1_scale.columns[1:])
print(df1_scale.columns[2])
print(type(df1_scale.columns[2]))
print(df1_scale.columns[2].find(","))
com_pos = df1_scale.columns[2].find(",")
print(df1_scale.columns[2][com_pos])
print(df1_scale.columns[2][com_pos+1:-1])


Index(['('block1_conv1', 0)', '('block1_conv1', 1)', '('block1_conv1', 2)',
       '('block1_conv1', 3)', '('block1_pool1', 0)', '('block1_pool1', 1)',
       '('block1_pool1', 2)', '('block1_pool1', 3)', '('block2_conv1', 0)',
       '('block2_conv1', 1)', '('block2_conv1', 2)', '('block2_conv1', 3)',
       '('block2_conv1', 4)', '('block2_conv1', 5)', '('block2_conv1', 6)',
       '('block2_conv1', 7)', '('block2_conv1', 8)', '('block2_conv1', 9)',
       '('block2_conv1', 10)', '('block2_conv1', 11)', '('block2_pool1', 0)',
       '('block2_pool1', 1)', '('block2_pool1', 2)', '('block2_pool1', 3)',
       '('block2_pool1', 4)', '('block2_pool1', 5)', '('block2_pool1', 6)',
       '('block2_pool1', 7)', '('block2_pool1', 8)', '('block2_pool1', 9)',
       '('block2_pool1', 10)', '('block2_pool1', 11)', '('before_softmax', 0)',
       '('before_softmax', 1)', '('before_softmax', 2)',
       '('before_softmax', 3)', '('before_softmax', 4)',
       '('before_softmax', 5)', '('before_so

In [0]:
df1_scale.columns[-1]

"('predictions', 9)"

In [0]:
import numpy as np
temp_x = 0
x = np.array([])
y = np.array([])
#for column in df1_scale.columns[1:]:
for column in df1_scale.columns[2:]:
  com_pos = column.find(",")
  #print(column[com_pos+1:-1])
  index = column[com_pos+1:-1]

  if int(index) == 0:
    print(column[2:com_pos-1])
    temp_x +=0.1
    num_index = int(index_before)
    print(num_index)
    for int_y in range(num_index+1):
#      print(int_y)
      temp_y = -int_y + (num_index-1)*0.5
      x = np.append(x, temp_x)
      y = np.append(y, temp_y)
      #print(y, temp_y)

  if column == df1_scale.columns[-1]:
    temp_x +=0.1
    num_index = int(index)
    print(num_index)
    for int_y in range(num_index+1):
      temp_y = -int_y + (num_index-1)*0.5
      x = np.append(x, temp_x)
      y = np.append(y, temp_y)

  index_before = index

print(x, y)



block1_pool1
3
block2_conv1
3
block2_pool1
11
before_softmax
11
predictions
9
9
[0.1 0.1 0.1 0.1 0.2 0.2 0.2 0.2 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3
 0.3 0.3 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.5 0.5 0.5 0.5
 0.5 0.5 0.5 0.5 0.5 0.5 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6] [ 1.  0. -1. -2.  1.  0. -1. -2.  5.  4.  3.  2.  1.  0. -1. -2. -3. -4.
 -5. -6.  5.  4.  3.  2.  1.  0. -1. -2. -3. -4. -5. -6.  4.  3.  2.  1.
  0. -1. -2. -3. -4. -5.  4.  3.  2.  1.  0. -1. -2. -3. -4. -5.]


In [0]:
%%time
import matplotlib.pyplot as plt
#plt.scatter(x,y, s=800)
for i in range(df4_scale.size):
  plt.figure(figsize=[12,10])
  plt.scatter(x,y, s=800, c=list(df4_scale.iloc[i][1:]), cmap= "Reds", edgecolor="k")
  plt.colorbar(orientation='horizontal', shrink=0.6)
  plt.axis("off")
  plt.savefig(output_dir[:-14]+ "/neuron_03_16/" + df4_scale.iloc[i][0] + ".png")

In [0]:
i = 3
df4_scale.iloc[i][0]

In [0]:
df4_scale.size